In [66]:
from scrape_headers import col_names

In [67]:
from all_pro_scrapper import all_pro_players

In [68]:
from all_players_scrapper import data_table

ImportError: cannot import name 'data_table' from 'all_players_scrapper' (c:\Users\siyun\OneDrive\Desktop\Python Scripts\Scrape-NFL-Combine\all_players_scrapper.py)

In [ ]:
import pandas as pd
df_all_players = pd.DataFrame(data_table, columns=col_names)
print(df_all_players)

df_all_pro_players = pd.DataFrame(all_pro_players, columns=col_names)
print(df_all_pro_players)

Empty DataFrame
Columns: [Year, Name, College, POS, Height (in), Weight (lbs), 40 Yard, Bench Press, Vert Leap (in), Broad Jump (in), Shuttle, 3Cone]
Index: []
